In [1]:
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from edc_appointment.models import Appointment

In [2]:
count = 0
dups = []
def update_window(g, pid):
    subject_identifier = pid

    print(">>>>>>>>>>>>>", pid)

    onses = g.objects.filter(subject_identifier=subject_identifier)
    
    for ons in onses:
        sn = ons.schedule_name
        
        try:
            ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
                                          visit_code_sequence=0,
                                          visit_code='2000M')
        except Appointment.MultipleObjectsReturned:
            dups.append(subject_identifier)
        else:

            ons.schedule_name = ons.schedule_name + 'x'
            ons.save()

            if sn[:-1] == 'x':
                count += 1
    #         import pdb; pdb.set_trace()
            try:
                ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
                                              schedule_name=sn,
                                              visit_code_sequence=0,
                                              visit_code='2000M')
            except Appointment.DoesNotExist:
                print(":::::::::::::::::", sn)

            ap0.schedule_name = ons.schedule_name
            ap0.save()

            # resave enroll visit
            cpe = CaregiverPreviouslyEnrolled.objects.get(subject_identifier=subject_identifier)

            cpe.save()

    #         import pdb; pdb.set_trace()
            print('}}}}}}}}}}}}', sn)
            ap1 = Appointment.objects.get(subject_identifier=subject_identifier,
                                          schedule_name=sn,
                                          visit_code_sequence=0,
                                         visit_code='2000M')

            ap1.user_modified = ap0.user_modified
            ap1.appt_status = ap0.appt_status
            ap1.appt_reason = ap0.appt_reason
            ap1.appt_datetime = ap0.appt_datetime

            ap1.save()

            try:
                v1 = MaternalVisit.objects.get(appointment=ap0)
                v1.appointment = ap1
                v1.save()
            except:
                pass

            ap0.delete()
            ons.delete()

In [3]:
##Maternal Window
from edc_appointment.models import Appointment

pids = []

for g in [OnScheduleCohortAEnrollment, OnScheduleCohortBEnrollment, OnScheduleCohortCEnrollment,
         OnScheduleCohortASec, OnScheduleCohortBSec, OnScheduleCohortCSec]:
    
    inner_pids = []
    for a in g.objects.all():
        try:
            prev_enrol = CaregiverPreviouslyEnrolled.objects.get(subject_identifier=a.subject_identifier,)
        except CaregiverPreviouslyEnrolled.DoesNotExist:
            pass
        else:
            if prev_enrol.report_datetime.date() != a.onschedule_datetime.date():
                pids.append([a.subject_identifier, g])
                inner_pids.append(a.subject_identifier)
            
    


In [11]:
pids

[['B142-040990044-5',
  flourish_caregiver.models.onschedule.OnScheduleCohortBEnrollment],
 ['B142-040990623-6',
  flourish_caregiver.models.onschedule.OnScheduleCohortCEnrollment],
 ['B142-040990602-0',
  flourish_caregiver.models.onschedule.OnScheduleCohortBSec],
 ['B142-040990612-9',
  flourish_caregiver.models.onschedule.OnScheduleCohortCSec],
 ['B142-040990002-3',
  flourish_caregiver.models.onschedule.OnScheduleCohortCSec],
 ['B142-040990001-5',
  flourish_caregiver.models.onschedule.OnScheduleCohortCSec],
 ['B142-040990005-6',
  flourish_caregiver.models.onschedule.OnScheduleCohortCSec],
 ['B142-040990003-1',
  flourish_caregiver.models.onschedule.OnScheduleCohortCSec],
 ['B142-040990606-1',
  flourish_caregiver.models.onschedule.OnScheduleCohortCSec],
 ['B142-040990007-2',
  flourish_caregiver.models.onschedule.OnScheduleCohortCSec]]

In [4]:
len(pids)

23

In [5]:
failed_pids = []
for pid, sch in pids:
        consents = SubjectConsent.objects.filter(subject_identifier=pid)
#         try:
#             FlourishConsentVersion.objects.get(screening_identifier=consents[0].screening_identifier)
#         except FlourishConsentVersion.DoesNotExist:
#             failed_pids.append(a.subject_identifier)
#         else:
        try:
            update_window(sch, pid)
        except Exception as e:
            print(e)
            failed_pids.append(pid)

>>>>>>>>>>>>> B142-040990504-8
}}}}}}}}}}}} a_enrol1_schedule1
>>>>>>>>>>>>> B142-040990581-6
}}}}}}}}}}}} a_enrol1_schedule1
>>>>>>>>>>>>> B142-040990006-4
}}}}}}}}}}}} b_enrol1_schedule1
>>>>>>>>>>>>> B142-040990009-8
}}}}}}}}}}}} b_enrol1_schedule1
>>>>>>>>>>>>> B142-040990044-5
}}}}}}}}}}}} b_enrol1_schedule1
Appointment matching query does not exist.
>>>>>>>>>>>>> B142-040990585-7
}}}}}}}}}}}} b_enrol1_schedule1
>>>>>>>>>>>>> B142-040990008-0
}}}}}}}}}}}} c_enrol1_schedule1
>>>>>>>>>>>>> B142-040990623-6
Associated caregiver consent on behalf of child for this participant not found
>>>>>>>>>>>>> B142-040990023-9
}}}}}}}}}}}} b_sec1_schedule1
}}}}}}}}}}}} b_sec2_schedule1
>>>>>>>>>>>>> B142-040990602-0
Associated caregiver consent on behalf of child for this participant not found
>>>>>>>>>>>>> B142-040990263-1
}}}}}}}}}}}} c_sec1_schedule1
>>>>>>>>>>>>> B142-040990612-9
Associated caregiver consent on behalf of child for this participant not found
>>>>>>>>>>>>> B142-040990002-3
}}}

In [1]:
count

NameError: name 'count' is not defined

In [9]:
len(failed_pids)

14

In [14]:

len(pids)

21

In [3]:
# for pid, sch in [['B142-040990044-5', OnScheduleCohortBEnrollment],]:

subject_identifier = 'B142-040990044-5'
onses = OnScheduleCohortCSec.objects.filter(subject_identifier='B142-040990044-5')
if onses.count() > 1:
    print("lllllll", subject_identifier)

for ons in onses:
    sn = ons.schedule_name

    ons.schedule_name = ons.schedule_name + 'x'
    ons.save()

    try:
        ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
                                      schedule_name=sn,
                                      visit_code_sequence=0,
                                      visit_code='2000M')
    except Appointment.DoesNotExist:
        print(":::::::::::::::::", sn)

    ap0.schedule_name = ons.schedule_name
    ap0.save()

     # resave enroll visit
    ccc = CaregiverChildConsent.objects.get(subject_identifier=ons.child_subject_identifier)

    ccc.save()

#         import pdb; pdb.set_trace()
    print('}}}}}}}}}}}}', sn)
    import pdb; pdb.set_trace()
    ap1 = Appointment.objects.get(subject_identifier=subject_identifier,
                                  schedule_name=sn,
                                  visit_code_sequence=0,
                                 visit_code='2000M')

    ap1.user_modified = ap0.user_modified
    ap1.appt_status = ap0.appt_status
    ap1.appt_reason = ap0.appt_reason
    ap1.appt_datetime = ap0.appt_datetime

    ap1.save()

    try:
        v1 = MaternalVisit.objects.get(appointment=ap0)
        v1.appointment = ap1
        v1.save()
    except:
        pass

    ap0.delete()
    ons.delete()


> /Users/imosweu/source/flourish-caregiver/flourish_caregiver/models/signals.py(259)caregiver_child_consent_on_post_save()
    257         import pdb; pdb.set_trace()
    258 
--> 259         if not children_count:
    260             children_count = 1 + child_dummy_consent_cls.objects.filter(
    261                 subject_identifier__startswith=instance.subject_consent.subject_identifier

ipdb> c
> /Users/imosweu/source/flourish-caregiver/flourish_caregiver/models/signals.py(259)caregiver_child_consent_on_post_save()
    257         import pdb; pdb.set_trace()
    258 
--> 259         if not children_count:
    260             children_count = 1 + child_dummy_consent_cls.objects.filter(
    261                 subject_identifier__startswith=instance.subject_consent.subject_identifier

ipdb> c
}}}}}}}}}}}} c_sec1_schedule1
> <ipython-input-3-290dcd46aa14>(33)<module>()
     31     print('}}}}}}}}}}}}', sn)
     32     import pdb; pdb.set_trace()
---> 33     ap1 = Appointment.objects

In [ ]:
subject_identifier = 'B142-040990044-5'
onses = OnScheduleCohortBEnrollment.objects.filter(subject_identifier='B142-040990044-5')
if onses.count() > 1:
    print("lllllll", subject_identifier)

for ons in onses:
    sn = ons.schedule_name

    ons.schedule_name = ons.schedule_name + 'x'
    ons.save()

    try:
        ap0 = Appointment.objects.get(subject_identifier=subject_identifier,
                                      schedule_name=sn,
                                      visit_code_sequence=0,
                                      visit_code='2000M')
    except Appointment.DoesNotExist:
        print(":::::::::::::::::", sn)

    ap0.schedule_name = ons.schedule_name
    ap0.save()

    # resave enroll visit
    ccc = CaregiverChildConsent.objects.get(subject_identifier=ons.child_subject_identifier)

    ccc.save()

#         import pdb; pdb.set_trace()
    print('}}}}}}}}}}}}', sn)
    import pdb; pdb.set_trace()
    ap1 = Appointment.objects.get(subject_identifier=subject_identifier,
                                  schedule_name=sn,
                                  visit_code_sequence=0,
                                 visit_code='2000M')

    ap1.user_modified = ap0.user_modified
    ap1.appt_status = ap0.appt_status
    ap1.appt_reason = ap0.appt_reason
    ap1.appt_datetime = ap0.appt_datetime

    ap1.save()

    try:
        v1 = MaternalVisit.objects.get(appointment=ap0)
        v1.appointment = ap1
        v1.save()
    except:
        pass

    ap0.delete()
    ons.delete()


In [22]:
ap1, ap2, ap3 = Appointment.objects.filter(subject_identifier='B142-040990044-5', visit_code='2000M')


In [24]:
ap3.__dict__

{'_state': <django.db.models.base.ModelState at 0x13470eaf0>,
 'created': datetime.datetime(2021, 5, 28, 13, 3, 42, 473363, tzinfo=<UTC>),
 'modified': datetime.datetime(2022, 3, 25, 13, 58, 56, 321574, tzinfo=<UTC>),
 'user_created': 'flourish',
 'user_modified': 'skgole',
 'hostname_created': 'flourish',
 'hostname_modified': 'Itumelengs-MacBook-Pro.local',
 'revision': '',
 'device_created': '99',
 'device_modified': '99',
 'id': UUID('5234ac74-98ae-44bf-9567-0ffc1ea8444a'),
 'site_id': 40,
 'subject_identifier': 'B142-040990044-5',
 'visit_schedule_name': 'c1_visit_schedule1',
 'schedule_name': 'c_sec1_schedule1',
 'visit_code': '2000M',
 'visit_code_sequence': 0,
 'timepoint_status': 'open',
 'timepoint_opened_datetime': datetime.datetime(2021, 5, 28, 13, 4, 5, tzinfo=<UTC>),
 'timepoint_closed_datetime': None,
 'timepoint': Decimal('0.0'),
 'timepoint_datetime': datetime.datetime(2021, 5, 28, 13, 4, 5, tzinfo=<UTC>),
 'appt_close_datetime': None,
 'facility_name': '5-day clinic',